In [5]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px
import plotly.graph_objects as go
from sklearn.preprocessing import (
    StandardScaler,
    MinMaxScaler,
    OneHotEncoder,
    PolynomialFeatures
)
from sklearn.metrics import (
    mean_squared_error,
    mean_absolute_error,
    r2_score,
    accuracy_score,
    precision_score,
    recall_score,
    f1_score,
    confusion_matrix,
    classification_report,
    make_scorer,
    roc_curve,
    auc
)
from sklearn.model_selection import (
    train_test_split,
    cross_validate,
    cross_val_score,
    GridSearchCV,
    KFold,
    learning_curve
)
from sklearn.linear_model import (
    LinearRegression,
    LogisticRegression,
    Ridge,
    Lasso
)
from sklearn.ensemble import (
    RandomForestRegressor,
    RandomForestClassifier
)
from sklearn.tree import DecisionTreeClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.multiclass import OneVsRestClassifier
from sklearn.pipeline import Pipeline, make_pipeline
from sklearn.compose import ColumnTransformer
from sklearn.impute import KNNImputer
from sklearn.decomposition import PCA
from imblearn.over_sampling import SMOTE
from imblearn.pipeline import Pipeline as ImbPipeline
from scipy import stats
import warnings
from sklearn.preprocessing import label_binarize
from sklearn.multiclass import OneVsRestClassifier
from sklearn.metrics import roc_curve, auc

In [95]:
db = pd.read_excel('../data/01_raw/Base_clientes_Monopoly.xlsx')

To begin with the exploratory analysis, it is essential to identify and understand the information contained in each of the datasets we will be using. Below is a summary of the three main datasets involved in our study.

In [98]:
db.head(8)

,Id,Subsegmento,Sexo,Region,Edad,Renta,Antiguedad,Internauta,Adicional,Dualidad,...,PagoNac_T01,PagoInt_T01,EeccNac_T01,EeccInt_T01,UsoL1_T01,UsoL2_T01,UsoLI_T01,IndRev_T01,target,NaN
0,1,160,M,13,43,NaN,130,1,1,0,...,33000,0,1099866,0,1099866,15080,0,R,0,NaN
1,2,160,H,13,46,143640,69,1,0,0,...,300000,0,214592,0,214592,83596,0,R,0,NaN
2,3,170,H,13,45,929106,24,1,1,0,...,216676,0,0,0,7400,0,0,T,0,NaN
3,4,151,H,13,46,172447,134,0,1,0,...,60000,0,272762,0,272762,10591,0,R,0,NaN
4,5,170,H,13,46,805250,116,0,1,1,...,272925,0,249562,0,75339,377782,0,R,0,NaN
5,6,170,H,13,47,707664,67,1,1,0,...,35800,0,35800,0,0,51197,0,R,0,NaN
6,7,811,H,13,48,1022833,21,1,0,1,...,9391,0,8818,0,8818,0,0,T,0,NaN
7,8,170,H,13,46,NaN,69,0,1,1,...,6000,0,283520,0,283520,0,0,R,0,NaN


In [97]:
db.columns = db.iloc[0]  

db = db[1:]


db.reset_index(drop=True, inplace=True)


print(db.columns)
print(db.head())   

Index([         'Id', 'Subsegmento',        'Sexo',      'Region',
              'Edad',       'Renta',  'Antiguedad',  'Internauta',
         'Adicional',    'Dualidad',
       ...
       'PagoNac_T01', 'PagoInt_T01', 'EeccNac_T01', 'EeccInt_T01',
         'UsoL1_T01',   'UsoL2_T01',   'UsoLI_T01',  'IndRev_T01',
            'target',           nan],
      dtype='object', name=0, length=575)
0 Id Subsegmento Sexo Region Edad   Renta Antiguedad Internauta Adicional  \
0  1         160    M     13   43     NaN        130          1         1   
1  2         160    H     13   46  143640         69          1         0   
2  3         170    H     13   45  929106         24          1         1   
3  4         151    H     13   46  172447        134          0         1   
4  5         170    H     13   46  805250        116          0         1   

0 Dualidad  ... PagoNac_T01 PagoInt_T01 EeccNac_T01 EeccInt_T01 UsoL1_T01  \
0        0  ...       33000           0     1099866           0 

In the data preparation process, the first row of the DataFrame `db` is assigned as the column names since the original Excel file contained numbers instead of descriptive names. This is crucial for facilitating the identification and handling of the data in subsequent analyses. Next, this first row is removed from the DataFrame, ensuring that only the relevant data remains for analysis and avoiding confusion from duplicate data. Following this, the index of the DataFrame is reset to keep it clean and organized, which is a good practice in data preparation. Finally, the new column names and the first few rows of the DataFrame are printed to verify that the operations have been executed correctly. This verification is essential in the CRISP-DM methodology to ensure that the data is properly structured before proceeding with exploratory analysis and modeling.

In [65]:
db.describe()

,NaN
count,0.0
mean,NaN
std,NaN
min,NaN
25%,NaN
50%,NaN
75%,NaN
max,NaN


In [57]:
db.isna().sum()


0
Id                 0
Subsegmento        0
Sexo               1
Region            53
Edad               0
               ...  
UsoL2_T01          0
UsoLI_T01          0
IndRev_T01         0
target             0
NaN            51124
Length: 575, dtype: int64

In [58]:
db.describe().T

,count,mean,std,min,25%,50%,75%,max
0,,,,,,,,
NaN,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [59]:
print(db.dtypes)

0
Id              object
Subsegmento     object
Sexo            object
Region          object
Edad            object
                ...   
UsoL2_T01       object
UsoLI_T01       object
IndRev_T01      object
target          object
NaN            float64
Length: 575, dtype: object


In [60]:
   print(len(db.columns))  # Esto te mostrará el número de columnas

575


In [61]:
conteo_filas = db.count()

# Mostrar el conteo de filas por columna
print(conteo_filas)


0
Id             51124
Subsegmento    51124
Sexo           51123
Region         51071
Edad           51124
               ...  
UsoL2_T01      51124
UsoLI_T01      51124
IndRev_T01     51124
target         51124
NaN                0
Length: 575, dtype: int64


In [62]:
# Contar el número total de filas en el DataFrame
numero_total_filas = db.shape[0]
print(f'Número total de filas: {numero_total_filas}')

Número total de filas: 51124


In [63]:
description = db.describe()
for column in description.columns:
    print(f"Analysis of {column}:")

    mean_value = description[column].mean(skipna=True)
    print(f" - Mean (Average): {mean_value:.2f} ")

    median_value = description[column].median(skipna=True)
    print(f" - Median: {median_value:.2f} ")

Analysis of nan:
 - Mean (Average): 0.00 
 - Median: 0.00 
